# Broker number of transaction past 12 months

#### Count for specific Broker Active

In [ ]:
CREATE OR REPLACE VIEW total_active AS 
SELECT DISTINCT
	cc_property_id,
	listing_office_name,
	state,
	status,
	COUNT(
		listing_office_name)
FROM
	mls_reza
WHERE
	date >= dateadd (
		DAY, - 365, getdate ())
GROUP BY
	listing_office_name,
	status,
	state,
	cc_property_id
HAVING
	status = 'Active'
	AND listing_office_name = 'The Lund Team, Inc'
ORDER BY
	cc_property_id DESC;

SELECT DISTINCT
	listing_office_name,
	COUNT(cc_property_id)
FROM
	total_active
GROUP BY
	listing_office_name


#### Count for specific Broker Pending

In [ ]:
CREATE OR REPLACE VIEW total_pending AS 
SELECT DISTINCT
	cc_property_id,
	listing_office_name,
	state,
	status,
	COUNT(
		listing_office_name)
FROM
	mls_reza
WHERE
	date >= dateadd (
		DAY, - 365, getdate ())
GROUP BY
	listing_office_name,
	status,
	state,
	cc_property_id
HAVING
	status = 'Pending'
	AND listing_office_name = 'The Lund Team, Inc'
ORDER BY
	cc_property_id DESC;

SELECT DISTINCT
	listing_office_name,
	COUNT(cc_property_id)
FROM
	total_active
GROUP BY
	listing_office_name

# Broker GMV of all transactions

#### Last 365 Days of Individual Brokers w/ Sale status & GMV for Average and Sum BY BROKERS

In [ ]:
-- Last 365 Days of Individual Brokers w/ Sale status & GMV AVG/SUM BY STATE

CREATE OR REPLACE VIEW gmvsale AS 
SELECT DISTINCT
	listing_office_name,
	state,
	status,
	COUNT(
		listing_office_name),
	price,
	cc_property_id,
	MAX(date)
FROM
	mls_reza
WHERE
	date >= dateadd (
		DAY, - 365, getdate ())
GROUP BY
	listing_office_name,
	cc_property_id,
	status,
	state,
	price
HAVING
	status = 'Sale'
	AND listing_office_name = 'Park Place Brokers'
ORDER BY
	listing_office_name,
	state ASC;

SELECT DISTINCT
	listing_office_name,
	AVG(price),
	SUM(price)
FROM
	gmvsale 
GROUP BY
	listing_office_name

# Broker 
#### % of Marketshare County

In [ ]:
CREATE OR REPLACE VIEW perc_marketshare_county AS 
SELECT DISTINCT
	listing_office_name,
	cc_property_address_county,
	state,
	status,
	COUNT(
		listing_office_name),
	price,
	cc_property_id,
	MAX(date)
FROM
	mls_reza
WHERE
	date >= dateadd (
		DAY, - 365, getdate ())
GROUP BY
	listing_office_name,
	cc_property_address_county,
	cc_property_id,
	status,
	state,
	price
HAVING
	status = 'Sale'
ORDER BY
	listing_office_name,
	state ASC;


SELECT DISTINCT
	listing_office_name,
	cc_property_address_county,
	AVG(price),
	SUM(price),
	SUM(price) * 100.0 / SUM(SUM(price)) OVER () AS Percentage
FROM
	perc_marketshare_county 
GROUP BY
	listing_office_name,
	cc_property_address_county
HAVING cc_property_address_county = 'LOS ANGELES'
ORDER BY 
	Percentage DESC;

#### % of Marketshare State

In [ ]:
-- Last 365 Days of Individual Brokers w/ Sale status for % of Marketshare. State

SELECT DISTINCT
	listing_office_name,
	state,
	AVG(price),
	SUM(price),
	SUM(price) * 100.0 / SUM(SUM(price)) OVER () AS Percentage
FROM
	perc_marketshare_county 
GROUP BY
	listing_office_name,
	state
HAVING state = 'CA'
ORDER BY 
	Percentage DESC;

# Broker 
#### of Agents Under Broker

In [ ]:
SELECT DISTINCT
	related_lastname_primary,
	related_lic_number,
	COUNT(lic_number)
FROM
	real_estate_agents
WHERE
	lic_type = 'Salesperson'
	AND related_lastname_primary = 'The Lund Team Inc'
GROUP BY
	related_lastname_primary,
	related_lic_number
ORDER BY
	COUNT DESC;

#### Top performing Agents

In [ ]:
# Top Performer List
CREATE OR REPLACE VIEW top_agents AS 
SELECT DISTINCT
	listing_office_name,
	listing_agent_name,
	state,
	status,
	COUNT(
		listing_agent_name),
	price,
	(cc_property_id),
	MAX(date)
FROM
	mls_reza
WHERE
	date >= dateadd (
		DAY, - 365, getdate ())
GROUP BY
	listing_office_name,
	listing_agent_name,
	cc_property_id,
	status,
	state,
	price
HAVING
	status = 'Sale';


SELECT DISTINCT
	listing_office_name,
	listing_agent_name,
	AVG(price) AS average,
	SUM(price) AS sum
FROM
	top_agents 
GROUP BY
	listing_office_name,
	listing_agent_name,
	status
ORDER BY 
	average DESC;

#### Dispersion Report

In [ ]:
#Dispersion Report
SELECT DISTINCT
	t.listing_office_name,
	t.listing_agent_name,
	t.price * 100 / gsum AS perc
FROM
	top_agents t
	JOIN (
		SELECT
			listing_office_name,
			SUM(price) AS gsum
		FROM
			mls_reza
		GROUP BY
			listing_office_name) tmp ON tmp.listing_office_name = t.listing_office_name
ORDER BY
	perc DESC

## Agent Performance

#### number of transactions last 12 months

In [ ]:
-- 1 number of transactions last 12 months

CREATE OR REPLACE VIEW agent_transaction_numbers AS 
SELECT DISTINCT
	listing_agent_name,
	listing_agent_license_number,
	status,
	COUNT(
		listing_agent_name),
	price,
	cc_property_id,
	MAX(date)
FROM
	mls_reza
WHERE
	date >= dateadd (
		DAY, - 365, getdate ())
GROUP BY
	listing_office_name,
	listing_agent_name,
	listing_agent_license_number,
	cc_property_id,
	status,
	price
HAVING
	status = 'Sale';


SELECT DISTINCT
	listing_agent_name,
	listing_agent_license_number,
	status,
	COUNT(listing_agent_name)
FROM
	agent_transaction_numbers
GROUP BY
	listing_agent_name,
	listing_agent_license_number,
	status
HAVING
	status = 'Sale'
ORDER BY
	COUNT DESC;


#### Avg transaction price

In [ ]:
-- 2 Avg transaction price
SELECT DISTINCT
	listing_agent_name,
	listing_agent_license_number,
	status,
	AVG(price),
	COUNT(listing_agent_name)
FROM
	agent_transaction_numbers
GROUP BY
	listing_agent_name,
	listing_agent_license_number,
	status
HAVING
	status = 'Sale'
ORDER BY
	COUNT DESC;

#### number of days to close from first active

In [ ]:
-- 3 number of days to close from first active
SELECT
	cc_property_id,
	listing_office_name,
	listing_agent_name,
	date
FROM
	mls_reza
LIMIT 100 

CREATE OR REPLACE VIEW prop AS
SELECT
	cc_property_id,
	cc_record_id,
	listing_office_name,
	listing_agent_name,
	status,
	date
FROM
	mls_reza
WHERE
	listing_agent_name = 'Garrett K. Lund'
	AND status = 'Sale'


SELECT
	*
FROM
	prop 
	
CREATE OR REPLACE VIEW transactions AS
	SELECT
		mls_reza.cc_property_id,
		mls_reza.listing_office_name,
		mls_reza.status,
		date(
			mls_reza.date),
		prop.date AS sales_date
	FROM
		mls_reza
		INNER JOIN prop ON mls_reza.cc_property_id = prop.cc_property_id
			AND lower(
				mls_reza.listing_office_name) = lower(
		prop.listing_office_name)
WHERE
	mls_reza.status <> 'Sale'
	
	
SELECT *
FROM transactions
		
CREATE OR REPLACE VIEW days_to_close AS		
SELECT
	cc_property_id,
	lower(listing_office_name) AS office_name,
	status,
	max(date),
	sales_date,
	date(sales_date) - max(date) AS Number_days_to_close
FROM
	transactions
GROUP BY
	cc_property_id,
	office_name,
	sales_date,
	status
HAVING
	status = 'Active'
	
SELECT office_name,
AVG(number_days_to_close)
FROM days_to_close
GROUP BY office_name

#### number of days to close from first pending

In [ ]:
-- 4 number of days to close from first pending
SELECT
	cc_property_id,
	listing_office_name,
	listing_agent_name,
	date
FROM
	mls_reza
LIMIT 100 

CREATE OR REPLACE VIEW prop AS
SELECT
	cc_property_id,
	cc_record_id,
	listing_office_name,
	listing_agent_name,
	status,
	date
FROM
	mls_reza
WHERE
	listing_agent_name = 'Garrett K. Lund'
	AND status = 'Sale'


SELECT
	*
FROM
	prop 
	
CREATE OR REPLACE VIEW transactions AS
	SELECT
		mls_reza.cc_property_id,
		mls_reza.listing_office_name,
		mls_reza.status,
		date(
			mls_reza.date),
		prop.date AS sales_date
	FROM
		mls_reza
		INNER JOIN prop ON mls_reza.cc_property_id = prop.cc_property_id
			AND lower(
				mls_reza.listing_office_name) = lower(
		prop.listing_office_name)
WHERE
	mls_reza.status <> 'Sale'
	
	
SELECT *
FROM transactions
		
CREATE OR REPLACE VIEW days_to_close AS		
SELECT
	cc_property_id,
	lower(listing_office_name) AS office_name,
	status,
	max(date),
	sales_date,
	date(sales_date) - max(date) AS Number_days_to_close
FROM
	transactions
GROUP BY
	cc_property_id,
	office_name,
	sales_date,
	status
HAVING
	status = 'Pending'
	
SELECT office_name,
AVG(number_days_to_close)
FROM days_to_close
GROUP BY office_name

#### Current listings

In [ ]:
CREATE OR REPLACE VIEW lastestpropevent AS
	SELECT DISTINCT
		cc_property_id,
		max(
			mls_reza.date) AS latest
	FROM
		mls_reza
	GROUP BY
		cc_property_id 
	
CREATE OR REPLACE VIEW lateststatus AS 
SELECT DISTINCT
		*
FROM (
		SELECT
			lastestevent.*,
			mls_reza.status,
			mls_reza.listing_agent_name,
			mls_reza.listing_office_name
		FROM
			lastestevent
				INNER JOIN mls_reza ON lastestevent.latest = mls_reza.date
			AND lastestevent.cc_property_id = mls_reza.cc_property_id
		WHERE status = 'Active'
		ORDER BY
			latest DESC)

#### Number of fallouts

In [ ]:
-- 6 Number of fallouts
CREATE OR REPLACE VIEW agent_fallout_numbers AS 
SELECT DISTINCT
	listing_agent_name,
	listing_agent_license_number,
	status,
	COUNT(
		listing_agent_name),
	price,
	cc_property_id,
	MAX(date)
FROM
	mls_reza
WHERE
	date >= dateadd (
		DAY, - 365, getdate ())
GROUP BY
	listing_office_name,
	listing_agent_name,
	listing_agent_license_number,
	cc_property_id,
	status,
	price
HAVING
	status != 'Active'
	AND status != 'Pending'
	AND status != 'Sale'
	AND status != 'Contingent'
LIMIT 100;


SELECT DISTINCT
	listing_agent_name,
	listing_agent_license_number,
	status,
	COUNT(listing_agent_name),
	AVG(price)
FROM
	agent_fallout_numbers
GROUP BY
	listing_agent_name,
	listing_agent_license_number,
	status
ORDER BY
	COUNT DESC;

#### Current Pendings

In [ ]:
CREATE OR REPLACE VIEW lastestpropevent AS
	SELECT DISTINCT
		cc_property_id,
		max(
			mls_reza.date) AS latest
	FROM
		mls_reza
	GROUP BY
		cc_property_id 
	
CREATE OR REPLACE VIEW lateststatus AS 
SELECT DISTINCT
		*
FROM (
		SELECT
			lastestevent.*,
			mls_reza.status,
			mls_reza.listing_agent_name,
			mls_reza.listing_office_name
		FROM
			lastestevent
				INNER JOIN mls_reza ON lastestevent.latest = mls_reza.date
			AND lastestevent.cc_property_id = mls_reza.cc_property_id
		WHERE status = 'Pending'
		ORDER BY
			latest DESC)

# Subject Property 
#### Original & Latest Price with Price Per Square Foot

In [ ]:
-- 8,9,10 Subject Property Original & Latest Price with Price Per Square Foot

SELECT DISTINCT
	cc_property_address_display_1,
	cc_property_address_display_2,
	listing_office_name,
	listing_agent_name,
	mls_number,
	status,
	FIRST_VALUE(price) OVER (PARTITION BY mls_number) AS Original_Price,
	LAST_VALUE(price) OVER (PARTITION BY mls_number) AS Current_Price,
	FIRST_VALUE(market_ppsf) OVER (PARTITION BY mls_number) AS Original_ppsf,
	LAST_VALUE(market_ppsf) OVER (PARTITION BY mls_number) AS Current_ppsf
FROM
	mls
GROUP BY
	cc_property_address_display_1,
	cc_property_address_display_2,
	listing_office_name,
	listing_agent_name,
	price,
	status,
	mls_number,
	market_ppsf
HAVING
	status = 'Active'
ORDER BY
	mls_number; 

### Property DNA

#### Listing History

In [ ]:
-- Number of times listed in the past 24 months.

SELECT DISTINCT
	cc_property_address_display_1,
	cc_property_address_display_2,
	zip,
	COUNT(cc_property_address_display_1)
FROM
	mls_reza
WHERE
	date >= dateadd (DAY, - 730, getdate ())
GROUP BY
	mls_number,
	status,
	cc_property_address_display_1,
	cc_property_address_display_2,
	zip
HAVING
	status = 'Active'
ORDER BY
	COUNT DESC; 

# Don't need to look at yet start and below  - Work in progress

#### Finding Counts for Agents with Brokers including Null


In [ ]:
-- Finding Counts for Agents with Brokers including Null

SELECT
    related_lastname_primary, COUNT(lic_type), lic_type
FROM
    real_estate_agents
GROUP BY
    lic_type, related_lastname_primary
HAVING lic_type = 'Salesperson'
ORDER BY
COUNT DESC; 

#### Finding Counts for Agents with Brokers without Null


In [ ]:
-- Finding Counts for Agents with Brokers not including Null

SELECT DISTINCT
    related_lastname_primary, COUNT(lic_type), lic_type
FROM
    real_estate_agents
GROUP BY
    lic_type, related_lastname_primary
HAVING lic_type = 'Salesperson' AND related_lastname_primary != ''
ORDER BY
COUNT DESC;  

#### Last 365 days of Individual Brokers w/ Active Listings in U.S.


In [ ]:
-- Last 365 of Individual Brokers w/ Active U.S.

SELECT DISTINCT
	listing_office_name, status, COUNT(listing_office_name)
FROM mls_reza
WHERE date >= dateadd(DAY, -365, getdate())
GROUP BY listing_office_name, status
HAVING status = 'Active'
ORDER BY COUNT DESC;

#### Last 365 days of Individual Brokers w/ Pending Listings in U.S.

In [ ]:
-- Last 12 Months of Individual Brokers w/ Pending

SELECT DISTINCT
	listing_office_name, status, COUNT(listing_office_name)
FROM mls_reza
WHERE date >= dateadd(DAY, -365, getdate())
GROUP BY listing_office_name, status
HAVING status = 'Pending'
ORDER BY COUNT DESC;

#### Last 365 Days of Individual Brokers w/ Active BY STATE

In [ ]:
-- Last 365 Days of Individual Brokers w/ Active BY STATE

SELECT DISTINCT
	listing_office_name, state, status, COUNT(listing_office_name)
FROM mls_reza
WHERE date >= dateadd(DAY, -365, getdate())
GROUP BY listing_office_name, status, state
HAVING status = 'Active'
ORDER BY COUNT DESC;

#### Last 365 Days of Individual Brokers w/ Pending BY STATE

In [ ]:
-- Last 365 Days of Individual Brokers w/ Pending BY STATE

SELECT DISTINCT
	listing_office_name, state, status, COUNT(listing_office_name)
FROM mls_reza
WHERE date >= dateadd(DAY, -365, getdate())
GROUP BY listing_office_name, status, state
HAVING status = 'Pending'
ORDER BY COUNT DESC;

#### Last 365 Days of Individual Brokers w/ ALL STATUS BY STATE

In [ ]:
-- Last 365 Days of Individual Brokers w/ ALL STATUS BY STATE

SELECT DISTINCT
	listing_office_name, state, status, COUNT(status)
FROM mls_reza
WHERE date >= dateadd(DAY, -365, getdate())
GROUP BY listing_office_name, state, status
ORDER BY listing_office_name, state ASC;

#### Last 365 days of Individual Brokers w/ Active BY COUNTY

In [ ]:
-- Last 365 Days of Individual Brokers w/ Active BY COUNTY

SELECT DISTINCT
	listing_office_name, cc_property_address_county, status, COUNT(listing_office_name)
FROM mls_reza
WHERE date >= dateadd(DAY, -365, getdate())
GROUP BY listing_office_name, status, cc_property_address_county
HAVING status = 'Active'
ORDER BY COUNT DESC;

#### Last 365 Days of Individual Brokers w/ Pending BY COUNTY

In [ ]:
-- Last 365 Days of Individual Brokers w/ Pending BY COUNTY

SELECT DISTINCT
	listing_office_name, cc_property_address_county, status, COUNT(listing_office_name)
FROM mls_reza
WHERE date >= dateadd(DAY, -365, getdate())
GROUP BY listing_office_name, status, cc_property_address_county
HAVING status = 'Pending'
ORDER BY COUNT DESC;

#### Last 365 Days of Individual Brokers w/ ALL STATUS BY COUNTY

In [ ]:
-- Last 365 Days of Individual Brokers w/ ALL STATUS BY COUNTY

SELECT DISTINCT
	listing_office_name, cc_property_address_county, status, COUNT(status)
FROM mls_reza
WHERE date >= dateadd(DAY, -365, getdate())
GROUP BY listing_office_name, cc_property_address_county, status
ORDER BY listing_office_name, cc_property_address_county ASC;

In [ ]:
-- Last 365 Days of Individual Brokers w/ Sale status for % of Marketshare. County

SELECT DISTINCT
	listing_office_name,
	cc_property_address_county,
	status,
	COUNT(listing_office_name),
	SUM(price),
	SUM(price) * 100.0 / SUM(SUM(price)) OVER () AS Percentage
FROM
	mls_reza
WHERE
	date >= dateadd (DAY, - 365, getdate ())
GROUP BY
	listing_office_name,
	cc_property_address_county,
	status
HAVING
	status = 'Sale'
ORDER BY
	Percentage DESC;

#### Last 365 Days of Individual Brokers w/ Sale status & GMV for Average and Sum BY COUNTY

In [ ]:
-- Last 365 Days of Individual Brokers w/ Sale status & GMV AVG/SUM BY COUNTY

SELECT DISTINCT
	listing_office_name,
	cc_property_address_county,
	status,
	COUNT(listing_office_name),
	AVG(price),
	SUM(price)
FROM
	mls_reza
WHERE
	date >= dateadd (DAY, - 365, getdate ())
GROUP BY
	listing_office_name,
	cc_property_address_county,
	status
HAVING
	status = 'Sale'
ORDER BY
	listing_office_name,
	cc_property_address_county ASC;

#### Last 365 Days of Individual Brokers w/ all status & GMV for Average and Sum BY STATE

In [ ]:
-- Last 365 Days of Individual Brokers w/ all status & GMV AVG/SUM BY STATE

SELECT DISTINCT
	listing_office_name,
	state,
	status,
	COUNT(listing_office_name),
	AVG(price),
	SUM(price)
FROM
	mls_reza
WHERE
	date >= dateadd (DAY, - 365, getdate ())
GROUP BY
	listing_office_name,
	state,
	status
ORDER BY
	listing_office_name,
	state ASC;

#### Last 365 Days of Individual Brokers w/ all status & GMV for Average and Sum BY COUNTY

In [ ]:
-- Last 365 Days of Individual Brokers w/ all status & GMV AVG/SUM BY COUNTY
SELECT DISTINCT
	listing_office_name,
	cc_property_address_county,
	status,
	COUNT(listing_office_name),
	AVG(price) AS AVERAGE_GMV,
	SUM(price) AS TOTAL_GMV
FROM
	mls_reza
WHERE
	date >= dateadd (DAY, - 365, getdate ())
GROUP BY
	listing_office_name,
	cc_property_address_county,
	status
ORDER BY
	listing_office_name,
	cc_property_address_county ASC;

In [ ]:
SELECT DISTINCT
    listing_office_name, state, status, COUNT(listing_office_name), LAST_VALUE(mls_number) OVER (PARTITION BY mls_number) AS alex
FROM
    mls_reza
WHERE record_date_time >= dateadd(DAY, -365, getdate())
GROUP BY
    listing_office_name, listing_agent_name, status, mls_number, state
HAVING status = 'Active'
ORDER BY COUNT DESC; 

SELECT t1.*
FROM mls_reza t1
WHERE t1.listing_office_name = (SELECT t2.listing_office_name
                 FROM mls_reza t2
                 WHERE t2.listing_office_name = t1.listing_office_name            
                 ORDER BY t2.listing_office_name DESC
                 LIMIT 1)

#### LAST Active and Pending Date Progress

In [ ]:
SELECT
	active.listing_office_name,
	MAX(active.record_date_time) AS active_date,
	active.address,
	active.status,
	pending.status
FROM
	mls_reza active
	INNER JOIN mls_reza pending ON pending.listing_office_name = active.listing_office_name
WHERE
	active.listing_office_name = 'The Lund Team, Inc'
	AND active.status = 'Active'
	AND pending.listing_office_name = 'The Lund Team, Inc'
	AND pending.status = 'Pending'
GROUP BY
	active.listing_office_name,
	active.address,
	active.status,
	pending.status
LIMIT 10;

#### LAST Active and Sale Date Progress

In [ ]:
SELECT
	active.listing_office_name,
	MAX(active.date) AS active_date,
	active.address,
	active.status,
	sold.status
FROM
	mls_reza active
	INNER JOIN mls_reza sold ON sold.listing_office_name = active.listing_office_name
WHERE
	active.listing_office_name = 'The Lund Team, Inc'
	AND active.status = 'Active'
	AND sold.listing_office_name = 'The Lund Team, Inc'
	AND sold.status = 'Sale'
GROUP BY
	active.listing_office_name,
	active.address,
	active.status,
	sold.status
LIMIT 10;

In [ ]:
SELECT DISTINCT
	listing_office_name, cc_property_address_county, status, COUNT(listing_agent_name)
FROM mls_reza
WHERE record_date_time >= dateadd(DAY, -365, getdate())
GROUP BY listing_office_name, status, cc_property_address_county, listing_agent_name
HAVING status = 'Active'
ORDER BY COUNT ASC;

In [ ]:
SELECT DISTINCT listing_office_name, cc_property_address_county,COUNT(listing_agent_name)
FROM mls
WHERE record_date_time >= dateadd(DAY, -365, getdate())
GROUP BY listing_office_name, cc_property_address_county
ORDER BY COUNT DESC;

In [ ]:
SELECT d.listing_office_name, COUNT(e.listing_agent_name) AS listing_agent_name
FROM mls d
LEFT JOIN mls e ON e.listing_office_name = d.listing_agent_name
GROUP BY d.listing_agent_name, d.listing_office_name

In [ ]:
SELECT DISTINCT
     cc_property_address_display_1, cc_property_address_display_2, zip, COUNT(cc_property_address_display_1)
FROM
    mls_reza
WHERE record_date_time >= dateadd(DAY, -730, getdate())
GROUP BY
    mls_number, status, cc_property_address_display_1, cc_property_address_display_2, zip
HAVING status = 'Active'
ORDER BY COUNT DESC; 

In [ ]:
##
SELECT DISTINCT listing_office_name, status, record_date_time, mls_number, cc_property_address_display_1
FROM mls_reza
WHERE listing_office_name LIKE 'The Lund Team, Inc' AND (status = 'Active' OR status = 'Pending')

In [ ]:
SELECT listing_office_name, MAX(record_date_time) AS active_date, mls_number 
FROM mls_reza 
WHERE listing_office_name LIKE 'The Lund Team, Inc' AND status = 'Active' 
GROUP BY mls_number, listing_office_name;


In [ ]:
SELECT active.listing_office_name, MAX(active.record_date_time) AS pending_date, active.address
FROM mls_reza active
INNER JOIN mls_reza pending ON pending.listing_office_name = active.listing_office_name
WHERE active.listing_office_name = 'The Lund Team, Inc' AND active.status = 'Active' AND pending.listing_office_name = 'The Lund Team, Inc' AND pending.status = 'Pending'
GROUP BY active.listing_office_name, active.address
LIMIT 10;
